# Tensor Renormalization Group

In [1]:
import numpy as np
import pyuni10 as uni10

In [2]:
ket = uni10.Bond(uni10.BD_IN, 2)
bra = uni10.Bond(uni10.BD_OUT, 2)
print(ket)
print(bra)

IN : (U1 = 0, P = 0, 0)|2, Dim = 2

OUT: (U1 = 0, P = 0, 0)|2, Dim = 2



In [3]:
T = uni10.UniTensorR([ket, ket, bra, bra])
T.SetName('T')
T.SetRawElem(np.array(range(16)))
print(T)


***************** T *****************
REAL(0)

             ____________
            |            |
        0___|2          2|___2    
            |            |   
        1___|2          2|___3    
            |            |   
            |____________|

================BONDS===============
IN : (U1 = 0, P = 0, 0)|2, Dim = 2
IN : (U1 = 0, P = 0, 0)|2, Dim = 2
OUT: (U1 = 0, P = 0, 0)|2, Dim = 2
OUT: (U1 = 0, P = 0, 0)|2, Dim = 2

===============BLOCKS===============
--- (U1 = 0, P = 0, 0): 
4 x 4 = 16 [ Actual Usage: 0 ], REAL

[ 
   0.       1.0000   2.0000   3.0000
   4.0000   5.0000   6.0000   7.0000
   8.0000   9.0000  10.0000  11.0000
  12.0000  13.0000  14.0000  15.0000
];
Total elemNum: 16





In [4]:
# decompose a rank-4 tensor T into two rank-3 tensor S
# current version assume all legs of T have the same dimension
def T2SS(T, D_cut):
    # S_LU and S_RD
    TM = T.GetBlock()
    D_cut = min(D_cut, TM.shape[0])
#     print(D_cut)
#     print('TM=\n',TM)
    U,Sigma,Vd = np.linalg.svd(TM)
    
    U_cut = U[:,:D_cut]
#     print('U_cut=\n',U_cut)
    Sigma_cut = Sigma[:D_cut]
#     print('Sigma_cut=\n',Sigma_cut)
    Vd_cut = Vd[:D_cut,:]
#     print('Vd_cut=\n',Vd_cut)

    bra = uni10.Bond(uni10.BD_OUT, D_cut)
    ket = uni10.Bond(uni10.BD_IN, D_cut)
 
    S_LU = uni10.UniTensorR([T.bond(0), T.bond(1), bra])
    S_LU.SetLabel([T.label(0),T.label(1),10])
#     print(S_LU)
    S_LU.PutBlock( U_cut @ np.diag(np.sqrt(Sigma_cut)))
    
    S_RD = uni10.UniTensorR([ket, T.bond(2), T.bond(3)])
    S_RD.SetLabel([10,T.label(2),T.label(3)])
    S_RD.PutBlock( np.diag(np.sqrt(Sigma_cut)) @ Vd_cut)

    # S_LD and S_RU
    # permute legs of T before SVD
#     T = uni10.Permute(T, [1,3,0,2], 2)
    uni10.Permute(T, [1,3,0,2], 2)
    TM = T.GetBlock()
#     print('TM=\n',TM)
    U,Sigma,Vd = np.linalg.svd(TM)
    
    U_cut = U[:,:D_cut]
#     print('U_cut=\n',U_cut)
    Sigma_cut = Sigma[:D_cut]
#     print('Sigma_cut=\n',Sigma_cut)
    Vd_cut = Vd[:D_cut,:]
#     print('Vd_cut=\n',Vd_cut)

    bra = uni10.Bond(uni10.BD_OUT, D_cut)
    ket = uni10.Bond(uni10.BD_IN, D_cut)
 
    S_LD = uni10.UniTensorR([T.bond(0), T.bond(1), bra])
    S_LD.SetLabel([T.label(0),T.label(1),10])
    S_LD.PutBlock( U_cut @ np.diag(np.sqrt(Sigma_cut)))
    
    S_RU = uni10.UniTensorR([ket, T.bond(2), T.bond(3)])
    S_RU.SetLabel([10,T.label(2),T.label(3)])
    S_RU.PutBlock( np.diag(np.sqrt(Sigma_cut)) @ Vd_cut)    

#     print(S_LU, S_RD)
    return S_LU, S_RD, S_LD, S_RU
    
S_LU, S_RD, S_LD, S_RU = T2SS(T,2)
S_LU.SetName('S_LU')
print(S_LU)
S_RD.SetName('S_RD')
print(S_RD)
S_LD.SetName('S_LD')
print(S_LD)
S_RU.SetName('S_RU')
print(S_RU)



SS_LURD = uni10.Contract(S_LU, S_RD)
print(SS_LURD)

SS_LDRU = uni10.Contract(S_LD, S_RU)
print(SS_LDRU)
print(uni10.Permute(SS_LDRU,[0,1,2,3],2))


**************** S_LU ****************
REAL(0)

             ____________
            |            |
        0___|2          2|___10   
            |            |   
        1___|2           |
            |            |   
            |____________|

================BONDS===============
IN : (U1 = 0, P = 0, 0)|2, Dim = 2
IN : (U1 = 0, P = 0, 0)|2, Dim = 2
OUT: (U1 = 0, P = 0, 0)|2, Dim = 2

===============BLOCKS===============
--- (U1 = 0, P = 0, 0): 
4 x 2 = 8 [ Actual Usage: 0 ], REAL

[ 
  -0.5444  -1.2548
  -1.8858  -0.6727
  -3.2272  -0.0907
  -4.5686   0.4913
];
Total elemNum: 8




**************** S_RD ****************
REAL(0)

             ____________
            |            |
       10___|2          2|___2    
            |            |   
            |           2|___3    
            |            |   
            |____________|

================BONDS===============
IN : (U1 = 0, P = 0, 0)|2, Dim = 2
OUT: (U1 = 0, P = 0, 0)|2, Dim = 2
OUT: (U1 = 0, P = 0, 0)|2, Dim = 2

=

In [5]:
def SSSS2T(S_RD, S_LD, S_RU, S_LU):
    # T 
    T = uni10.UniTensorR([S_RD.bond(0),S_RU.bond(0),S_LD.bond(2),S_LU.bond(2)])
    T.SetName('T')
    net = uni10.Network('SSSS2T.net')
    net.PutTensor('S_RD', S_RD)
    net.PutTensor('S_LD', S_LD)
    net.PutTensor('S_RU', S_RU)
    net.PutTensor('S_LU', S_LU)
    net.Launch(T)
    print(T)
    return 
    
SSSS2T(S_RD, S_LD, S_RU, S_LU)


************************************
REAL(0)

             ____________
            |            |
        0___|2          2|___2    
            |            |   
        1___|2          2|___3    
            |            |   
            |____________|

================BONDS===============
IN : (U1 = 0, P = 0, 0)|2, Dim = 2
IN : (U1 = 0, P = 0, 0)|2, Dim = 2
OUT: (U1 = 0, P = 0, 0)|2, Dim = 2
OUT: (U1 = 0, P = 0, 0)|2, Dim = 2

===============BLOCKS===============
--- (U1 = 0, P = 0, 0): 
4 x 4 = 16 [ Actual Usage: 0 ], REAL

[ 
 1012.9643 118.3749  -0.9757  22.3539
 -131.0654 -28.8362   1.6725  -3.7491
  71.2699   9.8802 -68.8580  -8.2872
  32.0536   3.9648  11.2845   3.4831
];
Total elemNum: 16



